In [1]:
!gpustat -f

gpu20                    Thu Mar 21 18:36:35 2024  515.105.01
[0] Tesla V100-DGXS-32GB | 44°C,   1 % |  5129 / 32768 MB | tanshiyin(1551M) tanshiyin(3575M)
 ├─ 240114 (   0%, 3179MB): /home/lr/tanshiyin/anaconda3/envs/py3.10/bin/python -m ipykernel_launcher -f /home/lr/tanshiyin/.local/share/jupyter/runtime/kernel-05577a2d-88fe-4f9c-8861-261eb8012622.json
 └─ 558468 ( 557%, 2970MB): python train_link_prediction.py --dataset_name mooc --model_name DyGFormer --load_best_configs --num_runs 5 --gpu 0 --same_timestamp_set False --pos_encoder_set True --fix_pos_encoder_set False --same_co_occurence False --comments position_encoding
[1] Tesla V100-DGXS-32GB | 45°C,  21 % |  7247 / 32768 MB | tanshiyin(3575M) tanshiyin(3575M)
 ├─ 558489 ( 941%, 2965MB): python train_link_prediction.py --dataset_name mooc --model_name DyGFormer --load_best_configs --num_runs 5 --gpu 1 --same_timestamp_set False --pos_encoder_set False --fix_pos_encoder_set False --same_co_occurence True --comments same_co_occu

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [17]:
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
    AutoModelForSeq2SeqLM,
)
from datasets import load_dataset, load_metric
import torch
from tqdm.notebook import tqdm

rouge = load_metric("rouge")

In [4]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"发现 {device_count} 个可用的 GPU:")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    device = torch.device('cuda')
else:
    print("未发现可用的 GPU")
    device = torch.device('cpu')

发现 1 个可用的 GPU:
GPU 0: Tesla V100-DGXS-32GB


First, we load the **Multi-news** dataset from huggingface dataset hub

In [7]:
dataset=load_dataset('multi_news')

In [6]:
download_model_dir = os.path.join(os.path.expanduser('~'), 'Code/NLP/model/download_model')

Then we load the fine-tuned PRIMERA model, please download [it](https://storage.googleapis.com/primer_summ/PRIMER_multinews.tar.gz) to your local computer.

In [10]:
# PRIMER_path = os.path.join(download_model_dir, 'PRIMER_multinews')
# print(PRIMER_path)
# PRIMER_path='../github/PRIMER/PRIMER_multinews_hf'
# PRIMER_path = 'allenai/PRIMERA'
PRIMER_path = 'allenai/PRIMERA-multinews'
TOKENIZER = AutoTokenizer.from_pretrained(PRIMER_path)
# MODEL = LEDForConditionalGeneration.from_pretrained(PRIMER_path)
MODEL = LEDForConditionalGeneration.from_pretrained(PRIMER_path).to(device)

MODEL.gradient_checkpointing_enable()
PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")
print(PAD_TOKEN_ID, DOCSEP_TOKEN_ID)

/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


1 50265


In [11]:
MODEL

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50266, 1024, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50266, 1024, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(4096, 1024)
      (layers): ModuleList(
        (0-11): 12 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
              (value_global): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): Linear(in_features=1024, out_features=1024, bias=True)

We then define the functions to pre-process the data, as well as the function to generate summaries.

In [19]:
import time

def process_document(documents):
    input_ids_all=[]
    for data in documents:
        all_docs = data.split("|||||")
        for i, doc in enumerate(all_docs):
            doc = doc.replace("\n", " ")
            doc = " ".join(doc.split())
            all_docs[i] = doc

        #### concat with global attention on doc-sep
        input_ids = []
        for doc in all_docs:
            input_ids.extend(
                TOKENIZER.encode(
                    doc,
                    truncation=True,
                    max_length=4096 // len(all_docs),
                )[1:-1]
            )
            input_ids.append(DOCSEP_TOKEN_ID)
        input_ids = (
            [TOKENIZER.bos_token_id]
            + input_ids
            + [TOKENIZER.eos_token_id]
        )
        input_ids_all.append(torch.tensor(input_ids))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids_all, batch_first=True, padding_value=PAD_TOKEN_ID
    )
    return input_ids


def batch_process(batch):
    input_ids=process_document(batch['document'])
    
    
    input_ids = input_ids.to(device)
    # get the input ids and attention masks together
    global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
    # put global attention on <s> token

    global_attention_mask[:, 0] = 1
    global_attention_mask[input_ids == DOCSEP_TOKEN_ID] = 1

    start_time = time.time()
    generated_ids = MODEL.generate(
        input_ids=input_ids,
        global_attention_mask=global_attention_mask,
        use_cache=False,
        max_length=1024,
        num_beams=1,
    )
    end_time = time.time()
    print("generate time: ", end_time - start_time)
    
    generated_str = TOKENIZER.batch_decode(
            generated_ids.tolist(), skip_special_tokens=True
        )

    result={}
    result['generated_summaries'] = generated_str
    result['gt_summaries']=batch['summary']
    return result

Next, we simply run the model on 10 data examples (or any number of examples you want)

In [23]:
import random
# data_idx = random.choices(range(len(dataset['test'])),k=10)
data_idx = range(len(dataset['test']))
dataset_small = dataset['test'].select(data_idx)
result_small = dataset_small.map(batch_process, batched=True, batch_size=2)

Map:   0%|          | 2/5622 [00:16<13:10:31,  8.44s/ examples]Input ids are automatically padded from 2837 to 3072 to be a multiple of `config.attention_window`: 512


generate time:  16.813998222351074


Map:   0%|          | 4/5622 [00:51<21:09:31, 13.56s/ examples]Input ids are automatically padded from 2840 to 3072 to be a multiple of `config.attention_window`: 512


generate time:  34.16968631744385


Map:   0%|          | 6/5622 [01:26<24:02:04, 15.41s/ examples]Input ids are automatically padded from 1153 to 1536 to be a multiple of `config.attention_window`: 512


...

Map: 100%|█████████▉| 5620/5622 [18:56:49<00:30, 15.13s/ examples]

generate time:  40.668569564819336


Map: 100%|██████████| 5622/5622 [18:57:10<00:00, 13.71s/ examples]

generate time:  20.72946286201477


Map: 100%|██████████| 5622/5622 [18:57:12<00:00, 12.14s/ examples]


After getting all the results, we load the evaluation metric. 


(Note in the original code, we didn't use the default aggregators, instead, we simply take average over all the scores.
We simply use 'mid' in this notebook)

In [26]:
rouge = load_metric("rouge")

/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [27]:
score=rouge.compute(predictions=result_small["generated_summaries"], references=result_small["gt_summaries"], use_stemmer=True)
print(score['rouge1'].mid)
print(score['rouge2'].mid)
print(score['rougeL'].mid)

Score(precision=0.5419884100177554, recall=0.47480144796544765, fmeasure=0.4947878147000231)
Score(precision=0.2140558341512011, recall=0.18871249610202792, fmeasure=0.19616786489854554)
Score(precision=0.2757042442467385, recall=0.24224741955292664, fmeasure=0.25193054225180933)


In [32]:
result_small['generated_summaries']

['– Amazon wants to be your go-to for last-minute deals this holiday season, so it\'s offering a little something extra for brick-and-mortar stores, TechCrunch reports. The online retailer is offering users of its Price Check app up to $5 off any product they buy at a brick-and-mortar store if they use the app to scan the barcode on the item, take a picture, or type in the product\'s name, the Verge reports. Amazon usually has lower prices than brick-and-mortar stores, "so this is just the cherry on top," one analyst says. "The ability to check prices on your mobile phone when you’re in a physical retail store is changing the way people shop," an Amazon mobile director says. "Price transparency means that you can save money on the products you want and that’s a great thing for customers. Price Check in-store deals are another incentive to shop smart this holiday season."',
 '– A day care in San Antonio has been shut down after an infant was allegedly bitten 27 times by another child, K

In [27]:
import random

In [30]:
random.choices(range(5000),k=5)

[4496, 1390, 2088, 2130, 1604]

– Facebook removed a photo of two men kissing in protest of a London pub’s decision to eject a same-sex couple for kissing, reports the America Blog. “Shares that contain nudity, or any kind of graphic or sexually suggestive content, are not permitted on Facebook,” the administrators of the Dangerous Minds Facebook page said in an email. The decision to remove the photo has prompted scores of people to post their own pictures of same-sex couples kissing in protest— dozens in the last few hours alone.

– Facebook has removed a photo from a protest page for a gay pub that booted a same-sex couple for kissing, USA Today reports. The Dangerous Minds Facebook page was trying to promote a “gay kiss-in” demonstration in London to protest the pub. The page used a photo of two men kissing to promote the event. But Facebook quickly removed the photo, saying in an email, “Shares that contain nudity, or any kind of graphic or sexually suggestive content, are not permitted on Facebook.” The decision to remove the photo has prompted scores of people to post their own pictures of same-sex couples kissing in protest— dozens in the last few hours alone.

## Evaluate for pytorch lighting model